In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("sample").master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)
spark.conf.set("spark.sql.shuffle.partitions","5")

## 이번 장 요약
- spark-submit 명령 : 운영용 애플리케이션 실행
- Dataset : 타입 안전성(type-safe, 타입 세이프)을 제공하는 구조적 API
- 구조적 스트리밍
- 머신러닝과 고급 분석
- RDD : 스파크의 저수준 API
- SparkR
- 서드파티 패키지 에코시스템

## 3.1 운영용 애플리케이션 실행하기
- spark-submit 명령 : 대화형 셸에서 개발한 프로그램을 운영용 애플리케이션으로 쉽게 전환할 수 있음
  - 애플리케이션 코드를 클러스터에 전송해 실행시키는 역할을 함
  - 클러스터에 제출된 애플리케이션은 작업이 종료되거나 에러가 발생할 때까지 실행됨
  - 스카프 애플리케이션 : 스탠드얼론, 메소스, YARN 클러스터 매니저를 이용해 실행됨
  - 해당 명령에 애플리케이션 실행에 필요한 자원과 실행 방식 그리고 다양한 옵션을 지정할 수 있음
  - 사용자는 스파크가 지원하는 프로그래밍 언어로 애플리케이션을 개발한 다음 실행할 수 있음
  - 로컬 머신에서 애플리케이션을 실행하는 것

In [2]:
./spark-3.1.2-bin-hadoop3.2/bin/spark-submit \
    --master local \
    ./spark-3.1.2-bin-hadoop3.2/examples/src/main/python/pi.py 10
    

SyntaxError: invalid syntax (<ipython-input-2-1aa252fbae24>, line 1)

- spark-submit 명령 중 master 옵션의 인숫값을 변경하면 스파크가 지원하는 스파크 스탠드얼론, 메소스 그리고 YARN 클러스터 매니저에서 동일한 애플리케이션을 실행할 수 있음
  - 해당 명령은 이 책의 예제를 실행하는 데 유용한 도구

## 3.2 Dataset : 타입 안정성을 제공하는 구조적 API
- 첫 번째로 설명할 API : Dataset
- Dataset 
  - 자바와 스칼라의 정적 데이터 타입에 맞는 코드, 즉 정적 타입 코드를 지원하기 위해 고안된 스파크의 구조적 API
  - 타입 안정성을 지원하며 동적 타입 언어인 파이썬과 R에서는 사용할 수 없음
  - 다양한 데이터 타입의 테이블형 데이터를 보관할 수 있는 Row 타입의 객체로 구성된 분산 컬렉션
  - DataFrame의 레코드를 사용자가 자바나 스칼라로 정의한 클래스에 할당하고 자바의 ArrayList 또는 스칼라의 Seq 객체 등의 고정 타입형 컬렉션으로 다룰 수 있는 기능을 제공함
  - **타입 안정성**을 지원하므로 초기화에 사용한 클래스 대신 다른 클래스를 사용해 접근할 수 있음
  - 다수의 소프트웨어 엔지니어가 잘 정의된 인터페이스 상호작용하는 대규모 애플리케이션을 개발하는 데 특히 유용함
  - 클래스(자바에서는 Dataset\<T>, 스칼라에서는 Dataset[T]로 표기함)는 내부 객체의 데이터 타입을 매개변수로 사용함
    - ex) Dataset[Person] : Person 클래스의 객체만 가질 수 있음
  - 스파크 2.0 버전에서는 자바의 JavaBean 패턴과 스칼라의 케이스 클래스 유형으로 정의된 클래스를 지원함
    - 이유 : 자동으로 타입 T를 분석한 다음 Dataset의 표 형식 데이터에 적합한 스키마를 생성해야 함
  - 장점
    - 필요한 경우에 선택적으로 사용할 수 있음
      - EX) 데이터 타입을 정의하고 map과 filter 함수를 사용할 수 잇음
  - 스파크는 처리를 마치고 결과를 DataFrame으로 자동 변환해 반환함
  - 스파크가 제공하는 여러 함수를 이용해 추가 처리 작업을 할 수 있음
    - 타입 안전성을 보장하는 코드에서 저수준 API를 사용할 수 있으며 고수준 API의 SQL를 사용해 빠른 분석을 할 수 있게 함
 

## 3.3 구조적 스트리밍
- 구조적 스트리밍 : 스파크 2.2 버전에서 안정화된 스트림 처리용 고수준 API
  - 사용 시, 구조적 API로 개발된 배치 모드의 연산을 스트리밍 방식으로 실행할 수 있고 지연 시간을 줄이고 증분 처리할 수 있음
  - 장점 
    - 배치 처리용 코드를 일부 수정하여 스트리밍 처리를 수행하고 값을 빠르게 얻을 수 있음
    - 프로토타입을 배치 잡으로 개발한 다음 스트리밍 잡으로 변환할 수 있으므로 개념 잡기가 수월함
    - 앞서 설명한 모든 작업은 데이터를 증분 처리하면서 수행됨

- 책에서 나온 파이썬 코드(local로 했을 땐 오래 걸리고 안 되서 전체 파일로 함)
  - staticDataFrame=spark.read_format("csv")\\
    
    .option("header","true")\\
    
    .option("inferSchema",true")\\
    
    .load("/data/retail-data/by-day/\*.csv")

In [2]:
#구조적 스트리밍 만들기 예제
#소매 데이터셋 : 특정 날짜와 시간 정보
# 하루치 데이터를 나타내는 by-day 디렉터리의 파일을 사용함
# 정적 데이터셋
staticDataFrame=spark.read.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("./Spark-The-Definitive-Guide-master/data/retail-data/all/online-retail-dataset.csv")

staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema=staticDataFrame.schema

- 시계열 데이터를 다루기 때문에 데이터를 그룹화하고 집계하는 방법을 알아볼 필요가 있음
  - 이를 위해 특정 고객(CustomerId로 구분함)이 대량으로 구매하는 영업 시간을 살펴볼 것임
  - ex) 총 구매비용 컬럼을 추가하고 고객이 가장 많이 소비한 날을 찾아보겠음
- 윈도우 함수
  - 집계 시에 시계열 컬럼을 기준으로 각 날짜에 대한 전체 데이터를 가지는 윈도우를 구성함
  - 윈도우는 간격을 통해 처리 요건을 명시할 수 있기 때문에 날짜와 타임스탬프 처리에 유용함
  - 스파크는 관련 날짜의 데이터를 그룹화함
  
  
- 정적 DataFrame 버전의 코드
  

In [3]:
from pyspark.sql.functions import window,col

staticDataFrame\
.selectExpr(
"CustomerID",
"(UnitPrice*Quantity) as total_cost",
"InvoiceDate")\
.groupBy(
col("CustomerID"),window(col("InvoiceDate"),"1 day"))\
.sum("total_cost")\
.show(5)

+----------+------+---------------+
|CustomerID|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+



- 로컬 모드로 이 코드를 실행하려면 로컬 모드에 적합한 셔플 파티션 수를 설정하는 것이 좋음
  - 셔플 파티션 수 : 셔플 이후에 생성될 파티션 수를 의미
  - 기본값 : 200

- 스트리밍 코드
  - 코드는 거의 바뀌지 않음
  - 차이점 : read 메서드 대신 readStream 메서드를 사용함
  - maxFilesPerTrigger 옵션을 추가로 지정함
    - 이 옵션을 사용해 한 번에 읽을 파일 수를 설정할 수 있음
    

In [4]:
#스트리밍답게 만드는 옵션이지만 운영 환경에 적용하는 것은 추천하지 않음
streamingDataFrame=spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger",1)\
    .format("csv")\
    .option("header","true")\
    .load("./Spark-The-Definitive-Guide-master/data/retail-data/all/online-retail-dataset.csv")

In [5]:
#스트리밍 유형인지 확인
streamingDataFrame.isStreaming

True

In [6]:
from pyspark.sql.functions import window,col
#기존 DataFrame 처리와 동일한 비즈니스 로직을 적용
# 총 판매 금액을 계산
purchaseByCustomerPerHour=streamingDataFrame\
    .selectExpr(
    "CustomerID",
    "(UnitPrice*Quantity) as total_cost",
    "InvoiceDate")\
    .groupBy(
    col("CustomerID"),window(col("InvoiceDate"),"1 day"))\
    .sum("total_cost")

- 위의 작업 역시 지연 연산으로 데이터 프로를 실행하기 위해 스트리밍 액션을 호출해야함
  - 스트리밍 액션은 어딘가에 데이터를 채워 넣어야 하므로 count 메서드와 같은 일반적인 정직 액션과는 조금 다른 특성을 가짐
  - 사용할 스트리밍 액션 : 트리거가 실행된 다음 데이터를 갱신하게 될 인메모리 테이블에 데이터를 저장함
  - 밑의 예제의 경우, 파일마다 트리거를 실행함
  - 스파크는 이전 집계값보다 더 큰 값이 발생한 경우에만 인메모리 테이블을 갱신하므로 언제가 가장 큰 값을 얻을 수 있음

In [7]:
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [8]:
#스트림이 시작되면 쿼리 실행 결과가 어떠한 형태로 
#인메모리 테이블에 기록되는지 확인할 수 있음
spark.sql("""
    SELECT *
    FROM customer_purchases
    ORDER BY 'sum(total_coust)' DESC
    """)\
    .show(5)

+----------+------+---------------+
|CustomerID|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+



- 더 많은 데이터를 읽을수록 테이블 구성이 바뀐다는 것을 알 수 있음
  - 각 파일에 있는 데이터에 따라 결과가 변경될 수도 있고 변경되지 않을 수도 있음
  - 상황에 따라 처리 결과를 콘솔에 출력할 수도 있음

In [9]:
purchaseByCustomerPerHour.writeStream\
    .format("console")\
    .queryName("customer_purchase_2")\
    .outputMode("complete")\
    .start()

- 사용한 두 가지 방식(메모리나 콘솔로 출력하는 방식과 파일별로 트리거를 수행하는 방식)을 운영 환경에서 사용하는 것은 좋지 않지만 구조적 스트리밍의 강력함을 편리하게 느껴보기에는 충분함
  - 스파크가 데이터를 처리하는 시점이 아닌 이벤트 시간에 따라 윈도우를 구성하는 방식에 주목할 필요가 있음
    - 이 방식을 사용하면 기존 스파크 스트리밍의 단점을 구조적 스트리밍으로 보완할 수 있음

## 

## 3.4 머신러닝과 고급 분석
- 스파크는 내장된 머신러닝 알고리즘 라이브러리인 MLlib을 사용해 대규모 머신러닝을 수행할 수 있음
  - MLlib을 사용하면 대용량 데이터를 대상으로 전처리, 멍잉, 모델학습 및 예측을 할 수 있음
     - 멍잉 : 데이터 랭글링이라고도 하며, 원본 데이터를 다른 형태로 변환한거나 매핑하는 과정을 의미
  - 구조적 스트리밍에서 예측하고자 할 때도 MLlib에서 학습시킨 다양한 예측 모델을 사용할 수 있음
- 스파크는 분류부터 회귀, 군집화, 그리고 딥러닝에 이르기까지 머신러닝과 관련된 정교한 API를 제공함
  - 머신러닝 API를 설명하기 위해 K-평균이라는 표준 알고리즘을 이용해 기본적인 군집화를 수행해보겠음
  - K평균이란?
    - 데이터에서 K개의 중심이 임의로 할당되는 군집화 알고리즘임
    - 중심점에 가까운 점들을 군집에 할당하고 할당된 점들의 중심을 계산함
    - 중심점을 센트로이드라고 함
    - 그런 다음 해당 센트로이드에 가장 가까운 점들의 군집에 레이블을 지정하고 새로 계산한 중심으로 센트로이드를 이동시킴
    - 이 과정을 정해진 횟수만큼 또는 수렴할 때까지(더는 중심 포인트의 변경 없을 때까지) 반복함
- 스파크는 데이터 전처리에 사용하는 다양한 메서드를 제공함

In [10]:
#원본 데이터를 올바른 포맷으로 만드는 트랜스포메이션을 정의
#실제로 모델을 학습한 다음 예측을 수행함
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



- MLlib의 머신러닝 알고리즘을 사용하기 위해서는 수치형 데이터가 필요함
  - 예제의 데이터는 타임스탬프, 정수 그리고 문자열 등 다양한 데이터 타입으로 이루어져 있으므로 수치형으로 변환해야 함

In [11]:
# 몇가지 DataFrame 트랜스포메이션을 사용해 날짜 데이터를 다루는 예제
from pyspark.sql.functions import date_format, col

preppedDataFrame=staticDataFrame\
    .na.fill(0)\
    .withColumn("day_of_week",date_format(col("InvoiceDate"),"EEEE"))\
    .coalesce(5)

- 데이터를 학습 데이터셋(학습셋)과 테스트 데이터셋(테스트셋)으로 분리해야 함
  - 예제에서는 특정 구매가 이루어진 날짜를 기준으로 직접 분리함
- MLlib의 트랜스포메이션 API(TrainValidationSplit이나 CrossValidator)를 사용해 학습 데이터셋과 테스트 데이터셋을 생성할 수 있음

In [12]:
trainDataFrame=preppedDataFrame\
    .where("InvoiceDate<'2011-07-01'")
testDataFrame=preppedDataFrame\
    .where("InvoiceDate>='2011-07-01'")

- 데이터가 준비되었으니 액션을 호출해 데이터를 분리함
  - 예제의 데이터는 시계열 데이터셋으로서 임의 날짜를 기준으로 데이터를 분리함
  - 비록 학습 데이터셋과 테스트 데이터셋을 분리하기 위한 최적의 방법은 아니지만 예제를 설명하기에는 충분함

In [13]:
#예제의 코드는 데이터셋을 대략 절반으로 나눔
trainDataFrame.count()
testDataFrame.count()

265596

- DataFrame의 트랜스포메이션 가능
- 스파크 MLlib은 일반적인 트랜스포메이션을 자동화하는 다양한 트랜스포메이션을 제공함
  - 그 중 하나가 바로 StringIndexer

In [14]:
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer()\
    .setInputCol("day_of_week")\
    .setOutputCol("day_of_week_index")

- 앞 예제는 요일을 수치형으로 반환함
  - 예를 들어 토요일을 6으로, 월요일을 1로 반환함
    - 이러한 번호 지정 체계는 수치로 표현되기 때문에 암묵적으로 토요일이 월요일보다 더 크다는 것을 의미함(잘못된 방식)
  - 문제점을 보완하기 위해서는 OneHotEncoder를 사용해 각 값을 자체 컬럼으로 인코딩해야 함
  - 이렇게 하면 특정 요일이 해당 요일인지 아닌지 불리언(boolean)타입으로 나타낼 수 있음

In [15]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder()\
    .setInputCol("day_of_week_index")\
    .setOutputCol("day_of_week_encoded")

- 위 예제의 결과
  - 벡터 타입을 구성할 컬럼 중 하나로 사용됨
- 스파크의 모든 머신러닝 알고리즘은 수치형 벡터 타입을 입력으로 사용함

In [16]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler=VectorAssembler()\
    .setInputCols(["UnitPrice","Quantity","day_of_week_encoded"])\
    .setOutputCol("features")

- 위 예제는 세 가지 핵심 특징인 가격, 수량, 특정 날짜의 요일을 가지고 있음
  - 다음은 나중에 입력값으로 들어올 데이터가 같은 프로세스를 거쳐 변환되도록 파이프라인을 설정하는 예제

In [17]:
from pyspark.ml import Pipeline

transformationPipeline=Pipeline()\
    .setStages([indexer,encoder,vectorAssembler])

- 학습 준비 과정 : 두 단계
  - 1. 우선 변환자를 데이터셋에 적합시켜야 함
    - 기본적으로 StringIndexer는 인덱싱할 고윳값의 수를 알아야 함
    - 고윳값의 수를 알 수 있다면 인코딩을 매우 쉽게 할 수 있지만 만약 알 수 없다면 컬럼에 있는 모든 고윳값을 조사하고 인덱싱해야 함

In [18]:
fittedPipeline=transformationPipeline.fit(trainDataFrame)

IllegalArgumentException: requirement failed: The input column day_of_week_index should have at least two distinct values.

- 2. 학습 데이터셋에 변호나자를 적합시키고 나면 학습을 위한 맞춤 파이프라인이 준비됨
  - 이것을 사용해서 일관되고 반복적인 방식으로 모든 데이터를 변환할 수 있음

In [19]:
transformedTraining=fittedPipeline.transform(trainDataFrame)

NameError: name 'fittedPipeline' is not defined

- 모델 학습에 사용할 파이프라인이 마련되었음
  - but, 데이터 캐싱을 설명하기 위해 파이프라인 구성 과정에서 데이터 캐싱 과정을 제외시켰음
  - 캐싱은 4단원에서 나옴
  - 동일한 트랜스포메이션을 계속 반복할 수 없으므로 그 대신 모델에 일부 하이퍼파라미터 튜닝값을 적용함
  - 캐싱을 사용하면 중간 변환된 데이터셋의 복사본을 메모리에 저장하므로 전체 파이프라인을 재실행하는 것보다 훨씬 빠르게 반복적으로 데이터셋에 접근할 수 있음

In [ ]:
transformedTraining.cache()

- 학습 데이터셋이 완성되었으므로 모델을 학습할 차례
  - 머신러닝 모델을 사용하려면 관련 클래스를 임포트하고 인스턴스를 생성해야 함

In [22]:
from pyspark.ml.clustering import KMeans

kmeans=KMeans().setK(20).setSeed(1)

- 스파크에서 머신러닝 모델을 학습시키는 과정(두 단계)
   1. 아직 학습되지 않은 모델을 초기화
   2. 해당 모델을 학습시킴
- MLlib의 DataFrame API에서 제공하는 모든 알고리즘은 항상 두 가지 유형으로 구성되어 있으며 다음과 같은 명명규칙을 따름
  - 학습 전 알고리즘 명칭 : Algorithm   ->>>>> KMeans
  - 학습 후 알고리즘 명칭 : AlgorithmModel ->>>>>> KMeansModel
- MLlib의 DataFrame API에서 제공하는 추정자는 앞서 사용한 전처리 변환자(예 : StringIndexer)와 거의 동일한 인터페이스를 가지고 있음
  - 이 인터페이스를 사용해 전체 파이프라인의 학습 과정을 단순화할 수 있음
  - 이 장의 예제에서는 단계별로 설명하기 위해 파이프라인에서 kmeans 모델 설정 과정을 생략하겠음

In [ ]:
kmModel=kmeas.fit(transformedTraining)

- 모델 학습이 완료되면 몇 가지 성과 평가지표에 따라 학습 데이터셋에 대한 비용을 계산할 수 있음
  - 예제에서 사용한 데이터셋의 군집 비용은 상당히 높은 편
    - 입력 데이터에 대한 전처리와 표준화 작업이 적절히 이루어지지 않았기 때문

In [ ]:
kmModel.computeCost(transformedTraining)

transformedTest=fittedPipeline.transform(testDataFrame)
kmModel.computeCost(transformedTest)

## 3.5 저수준 API
- 스파크는 RDD를 통해 자바와 파이썬 객체를 다루는 데 필요한 다양한 기본 기능(저수준 API)을 제공함
  - 스파크의 거의 모든 기능은 RDD를 기반으로 만들어졌음
  - DataFrame 연산도 RDD를 기반으로 만들어졌으며 편리하고 매우 효율적인 분산 처리를 위해 저수준 명령으로 컴파일됨
  - 원시 데이터를 읽거나 다루는 용도로 RDD를 사용할 수 있지만 대부분은 구조적 API를 사용하는 것이 좋음
    - 하지만 RDD를 이용해 파티션과 같은 물리적 실행 특성을 결정할 수 있으므로 DataFrame보다 더 세밀한 제어를 할 수 있음

- 드라이버 시스템의 메모리에 저장된 원시 데이터를 병렬처리하는데 RDD를 사용할 수 있음


In [23]:
# 간단한 숫자를 이용해 병렬화해 RDD를 생성하는 예제
# 그런 다음 다른 DataFrame과 함께 사용할 수 있도록 DataFrame으로 변환함
from pyspark.sql import Row
spark.sparkContext.parallelize([Row(1),Row(2),Row(3)]).toDF()

_1
1
2
3


- RDD는 스칼라뿐만 아니라 파이썬에서도 사용할 수 있음
  - 하지만 두 언어의 RDD가 동일하진 않음
  - 언어와 관계없이 동일한 실행 특성을 제공하는 DataFrame API와는 다르게 RDD는 세부 구현 방식에서 차이를 보임
  - 낮은 버전의 스파크 코드를 계속 사용해야 하는 상황이 아니라면 RDD를 사용해 스파크 코드를 작성할 필요는 없음
  
- 최신 버전의 스파크에서는 기본적으로 RDD를 사용하지 않지만 비정형 데이터나 정제되지 않은 원시 데이터를 처리해야 한다면 RDD를 사용해야 함

## 3.7 스파크의 에코시스템과 패키지
- 스파크가 자랑하는 최고 장점 : 커뮤니티가 만들어낸 패키지 에코시스템과 다양한 기능
- 스파크 패키지 목록 : spark-packages.org